# 任务 4：部署用于异步推理的模型

## 任务 4.1：环境设置

安装程序包和依赖项。

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from botocore.exceptions import ClientError

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

将来自训练和优化实验的模型保存在默认 Amazon Simple Storage Service (Amazon S3) 存储桶中。使用 **create_model** 设置模型，并配置 **ModelDataUrl** 以引用经过训练的模型。

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

将异步记录上传到默认 Amazon S3 存储桶。

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="asynchronous_records.csv", Bucket=bucket, Key=f"{prefix}/asynchronous_records.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
input_location = f"s3://{bucket}/{prefix}/asynchronous_records.csv"

## 任务 4.2：从提供的经过合成和重新训练的模型创建终端节点

Amazon SageMaker Asynchronous Inference 是 SageMaker 中的一项功能，用于对传入的请求进行排队和异步处理。此功能非常适合负载较大（最多 1GB）、处理时间较长（最多 15 分钟）且需要实现近乎实时延迟的请求。利用 Asynchronous Inference，您可以在没有请求要处理时自动将实例数量减少为零来降低成本。因此，您只需在终端节点处理请求时付费。

使用 SageMaker Python SDK 创建异步终端节点分三个步骤。这些步骤与用于实时和无服务器终端节点的步骤相同，但两者的配置不同：
1.在 SageMaker 中创建 SageMaker 模型。
2.为 HTTPS 终端节点创建终端节点配置。
3.创建 HTTPS 终端节点。

您已创建模型。现在，您可以创建终端节点配置和终端节点。

首先，设置终端节点配置名称和要使用的实例类型。然后，调用 CreateEndpointConfig API。

要创建终端节点配置，您需要设置以下选项：
- **VariantName**：生产变体（生产环境中的一个或多个模型）的名称。
- **ModelName**：您要托管的模型的名称。这是您在创建模型时指定的名称。
- **InstanceType**：计算实例类型。
- **S3OutputPath**：当请求中未提供位置时，要将响应输出上传到的位置。
- **MaxConcurrentInvocationsPerInstance**：（可选）SageMaker 客户端发送到模型容器的最大并发请求数。

或者，您也可以设置 NotificationConfig，选择 Amazon Simple Notification Service (Amazon SNS) 主题以在推理请求成功或失败时发布通知。在本实验中，您无需设置此选项。

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-asynchronous-endpoint-{}'.format(create_date)                              
output_location = f"s3://{bucket}/{prefix}/output"

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant.
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
            
        } 
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": output_location
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker chooses an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

接下来，创建终端节点。当您创建异步终端节点时，SageMaker 会启动机器学习 (ML) 计算实例，并按照配置中指定的方式部署模型。有关可用于异步终端节点的选项的更多信息，请参阅 [Asynchronous Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html)。

当终端节点正在运行时，辅助函数将打印终端节点 Amazon Resource Name (ARN)。终端节点可能需要长达 7 分钟的时间才能完成创建。

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


在 SageMaker Studio 中，您可以在 **Endpoints**（终端节点）选项卡下查看终端节点详细信息。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **asynchronous_inference.ipynb** 选项卡拖到一边，或者选择（右键单击）**asynchronous_inference.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览终端节点时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完终端节点后，通过选择 **asynchronous_inference.ipynb** 选项卡返回至笔记本。

1.选择 **SageMaker Home**（SageMaker 主页）图标。
2.选择 **Deployments**（部署）。
3.选择 **Endpoints**（终端节点）。

此时 SageMaker Studio 将显示 **Endpoints**（终端节点）选项卡。

4.选择 **Name**（名称）列中包含 **income-model-asynchronous-** 的终端节点。

如果终端节点未出现，请选择刷新图标，直到终端节点出现在列表中。

此时 SageMaker Studio 将显示 **ENDPOINT DETAILS**（终端节点详细信息）选项卡。

5.选择 **AWS settings**（AWS 设置）选项卡。

如果您在终端节点完成创建之前打开了终端节点，请选择刷新图标，直到 **Endpoint status**（终端节点状态）从 *Creating*（正在创建）变为 *InService*（正在运行）。

**Endpoint type**（终端节点类型）显示为 **Async**（异步）。**Endpoint runtime settings**（终端节点运行时设置）部分会显示您之前在笔记本中选择的配置。

## 任务 4.3：使用异步客户记录调用终端节点以进行异步推理

使用 SageMaker 托管服务部署模型后，您可以向终端节点发送测试数据，以便在其上测试模型。

要测试异步终端节点，您必须在 API 调用中包含 Amazon S3 输入位置。对于本实验，默认 SageMaker S3 存储桶中有一个 asynchronous_records.csv 文件，其中包含 100 条客户记录，您可以用其测试终端节点。如果操作成功，服务将发回 HTTP 202 响应。

In [ ]:
#send-test-file
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)

print(response)

output_key = response['OutputLocation'].split("/", 3)[3]
print('\nThe output key is: {}'.format(output_key))

检查输出位置以了解推理是否已处理。如果已处理，则打印出调用中包含的所有客户的预测分数。

In [ ]:
#get-output
def get_output():
    while True:
        try:
            return sagemaker.session.Session().read_s3_file(bucket=bucket, key_prefix=output_key)
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("Waiting for output...")
                time.sleep(2)
                continue
            raise

output = get_output()
print(f"Predictions for the 100 customers: {output}")

## 任务 4.4：删除终端节点

清理终端节点可以分三个步骤完成。首先，删除终端节点。然后，删除终端节点配置。最后，如果不再需要部署的模型，则将其删除。

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 总结

恭喜！ 您已使用 SageMaker 成功创建一个异步终端节点，并使用 SageMaker Python SDK 调用了该终端节点。

本实验的下一项任务侧重于批量转换。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续执行**任务 5：使用批量转换从大型数据集获取推理**。